In [2]:
# Import necessary libraries (taken from the multi_analysis script)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import mne
from scipy import signal, stats
import pandas as pd
import logging
from typing import Dict, List, Tuple, Optional
from matplotlib.gridspec import GridSpec
import matplotlib.patches as mpatches
from mpl_toolkits.axes_grid1 import make_axes_locatable
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm
import json
import warnings

# Import custom modules
from imagined_vs_actual_analysis_new import ImaginedVsActualAnalyzer, COLORS

# **Code for calculating Power Spectral Entropy Differences between Imagined and Actual Movements**
Power Spectral Entropy (PSE) measures the complexity or "peakiness" of a signal’s power spectral density (PSD) by applying Shannon entropy to the frequency domain. A flat, complex signal like white noise yields high entropy (1), while a simple signal with sharp peaks (0) has low entropy. People with lower power spectral entropy are easier for a BCI to read and understand since they have clearer signals.



In [3]:
def spectral_entropy(psd, eps=1e-12):
    # Compute spectral entropy from power spectral density (psd) vector. eps to avoid log(0).
    psd = psd + eps
    # Converts psd to a probability distribution
    psd /= psd.sum()
    # Compute spectral entropy
    H = -np.sum(psd * np.log2(psd))
    # Normalize by log2 of number of frequency bins to range [0,1]
    return H / np.log2(len(psd))

def entropy_from_epochs(
    epochs: mne.Epochs,
    fmin: float = 8.0,
    fmax: float = 30.0
) -> np.ndarray:
    # Compute mean spectral entropy per epoch across channels.
    psd = epochs.compute_psd(
        method="welch",
        fmin=fmin,
        fmax=fmax,
        average="mean",
        verbose=False
    )
    # Shape: (n_epochs, n_channels, n_freqs)
    psds = psd.get_data()
    entropies = np.zeros((psds.shape[0], psds.shape[1]))
    for e in range(psds.shape[0]):
        for ch in range(psds.shape[1]):
            entropies[e, ch] = spectral_entropy(psds[e, ch])

    # Return mean entropy per epoch (averaged across channels)
    return entropies.mean(axis=1)

def entropy_modulation(task_epochs, baseline_raw):
    baseline_epochs = mne.make_fixed_length_epochs(
        baseline_raw,
        duration=2.0,
        overlap=1.0,
        preload=True,
        verbose=False
    )
    H_task = entropy_from_epochs(task_epochs).mean()
    H_base = entropy_from_epochs(baseline_epochs).mean()
    return H_base - H_task  # Positive = more structured during task



# Test run on 1 person 
analyzer = ImaginedVsActualAnalyzer(subject_id="S003")
analyzer.load_and_preprocess_data()
baseline_raw = analyzer.data['baseline_eyes_open']['raw']
task_key = "Left/Right Fist 1_imagined"
task_epochs = analyzer.data[task_key]['epochs'][:10]
entropy_score = entropy_modulation(task_epochs, baseline_raw)
print(f"Motor imagery entropy modulation: {entropy_score:.3f}")


2026-01-20 02:28:12,317 - INFO - ======================================================================
2026-01-20 02:28:12,318 - INFO - LOADING DATA FOR S003 - IMAGINED VS ACTUAL ANALYSIS
2026-01-20 02:28:12,318 - INFO - ======================================================================
2026-01-20 02:28:12,318 - INFO - 
Loading baseline: eyes_open (Run 1)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


2026-01-20 02:28:12,379 - INFO -   ✓ Loaded baseline: 61.0s, 3 channels
2026-01-20 02:28:12,380 - INFO - 
Loading baseline: eyes_closed (Run 2)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


2026-01-20 02:28:12,394 - INFO -   ✓ Loaded baseline: 61.0s, 3 channels
2026-01-20 02:28:12,394 - INFO - 
Loading Left/Right Fist 1_real (Run 3)


...

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


2026-01-20 02:28:12,644 - INFO -   ✓ Loaded: 29 epochs, 3 channels
2026-01-20 02:28:12,644 - INFO - 
Loading Fists/Feet 3_imagined (Run 14)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


2026-01-20 02:28:12,664 - INFO -   ✓ Loaded: 29 epochs, 3 channels
2026-01-20 02:28:12,664 - INFO - 
✓ Successfully loaded 14 datasets


Motor imagery entropy modulation: -0.053


In [4]:
def spectral_entropy(psd, eps=1e-12):
    # Compute spectral entropy from power spectral density (psd) vector. eps to avoid log(0).
    psd = psd + eps
    # Converts psd to a probability distribution
    psd /= psd.sum()
    # Compute spectral entropy
    H = -np.sum(psd * np.log2(psd))
    # Normalize by log2 of number of frequency bins to range [0,1]
    return H / np.log2(len(psd))



def band_entropy_from_epochs(
    epochs: mne.Epochs,
    fmin: float,
    fmax: float
) -> np.ndarray:
    # Compute mean spectral entropy per epoch across channels.
    psd = epochs.compute_psd(
        method="welch",
        fmin=fmin,
        fmax=fmax,
        average="mean",
        verbose=False
    )
    # Shape: (n_epochs, n_channels, n_freqs)
    psds = psd.get_data()
    entropies = np.zeros((psds.shape[0], psds.shape[1]))
    for e in range(psds.shape[0]):
        for ch in range(psds.shape[1]):
            entropies[e, ch] = spectral_entropy(psds[e, ch])

    # Return mean entropy per epoch (averaged across channels)
    return entropies.mean(axis=1)



def entropy_modulation_band(
    task_epochs: mne.Epochs,
    baseline_raw: mne.io.BaseRaw,
    fmin: float,
    fmax: float
) -> float:
    # Positive value = entropy decreases during task
    baseline_epochs = mne.make_fixed_length_epochs(
        baseline_raw,
        duration=2.0,
        overlap=1.0,
        preload=True,
        verbose=False
    )
    H_task = band_entropy_from_epochs(task_epochs, fmin, fmax).mean()
    H_base = band_entropy_from_epochs(baseline_epochs, fmin, fmax).mean()
    return H_base - H_task



def imagined_real_entropy_gap(
    analyzer: ImaginedVsActualAnalyzer,
    band: str = "beta",
    max_epochs: int = 20
) -> Dict[str, float]:
    # Computes entropy modulation for imagined and real tasks and returns their gap per task.
    bands = {
        "alpha": (8, 13),
        "beta": (13, 30)
    }
    fmin, fmax = bands[band]
    baseline_raw = analyzer.data['baseline_eyes_open']['raw']
    results = {}

    for pair in analyzer.task_pairs:
        real_key = f"{pair['name']}_real"
        imag_key = f"{pair['name']}_imagined"
        if real_key not in analyzer.data or imag_key not in analyzer.data:
            continue
        real_epochs = analyzer.data[real_key]['epochs'][:max_epochs]
        imag_epochs = analyzer.data[imag_key]['epochs'][:max_epochs]
        if len(real_epochs) < 5 or len(imag_epochs) < 5:
            continue
        ΔH_real = entropy_modulation_band(
            real_epochs, baseline_raw, fmin, fmax
        )
        ΔH_imag = entropy_modulation_band(
            imag_epochs, baseline_raw, fmin, fmax
        )
        results[pair['name']] = {
            "ΔH_real": ΔH_real,
            "ΔH_imagined": ΔH_imag,
            "gap": ΔH_real - ΔH_imag
        }
    return results



# Test run on 1 person 
analyzer = ImaginedVsActualAnalyzer(subject_id="S026")
analyzer.load_and_preprocess_data()
beta_results = imagined_real_entropy_gap(
    analyzer,
    band="beta"
)
alpha_results = imagined_real_entropy_gap(
    analyzer,
    band="alpha"
)
print("BETA BAND RESULTS")
for k, v in beta_results.items():
    print(k, v)
print("\nALPHA BAND RESULTS")
for k, v in alpha_results.items():
    print(k, v)

2026-01-20 02:28:12,683 - INFO - ======================================================================
2026-01-20 02:28:12,684 - INFO - LOADING DATA FOR S026 - IMAGINED VS ACTUAL ANALYSIS
2026-01-20 02:28:12,684 - INFO - ======================================================================
2026-01-20 02:28:12,685 - INFO - 
Loading baseline: eyes_open (Run 1)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


2026-01-20 02:28:12,702 - INFO -   ✓ Loaded baseline: 61.0s, 3 channels
2026-01-20 02:28:12,702 - INFO - 
Loading baseline: eyes_closed (Run 2)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


2026-01-20 02:28:12,717 - INFO -   ✓ Loaded baseline: 61.0s, 3 channels
2026-01-20 02:28:12,717 - INFO - 
Loading Left/Right Fist 1_real (Run 3)


...

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


2026-01-20 02:28:12,972 - INFO -   ✓ Loaded: 29 epochs, 3 channels
2026-01-20 02:28:12,972 - INFO - 
Loading Fists/Feet 3_imagined (Run 14)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


2026-01-20 02:28:12,994 - INFO -   ✓ Loaded: 29 epochs, 3 channels
2026-01-20 02:28:12,994 - INFO - 
✓ Successfully loaded 14 datasets


BETA BAND RESULTS
Left/Right Fist 1 {'ΔH_real': np.float64(-0.05632216811942248), 'ΔH_imagined': np.float64(-0.04440606481395204), 'gap': np.float64(-0.011916103305470438)}
Fists/Feet 1 {'ΔH_real': np.float64(-0.05268018303800437), 'ΔH_imagined': np.float64(-0.03961218743010031), 'gap': np.float64(-0.01306799560790406)}
Left/Right Fist 2 {'ΔH_real': np.float64(-0.04462554798475804), 'ΔH_imagined': np.float64(-0.04717810266333344), 'gap': np.float64(0.0025525546785754027)}
Fists/Feet 2 {'ΔH_real': np.float64(-0.04161916441451252), 'ΔH_imagined': np.float64(-0.033499669157803336), 'gap': np.float64(-0.008119495256709186)}
Left/Right Fist 3 {'ΔH_real': np.float64(-0.04205094965667866), 'ΔH_imagined': np.float64(-0.04548955275438571), 'gap': np.float64(0.003438603097707049)}
Fists/Feet 3 {'ΔH_real': np.float64(-0.04122653841187429), 'ΔH_imagined': np.float64(-0.04528720055633084), 'gap': np.float64(0.0040606621444565505)}

ALPHA BAND RESULTS
Left/Right Fist 1 {'ΔH_real': np.float64(-0.0698

## **Code for Calculating Brainwave Signal Lempel-Ziv Complexity**

Lempel-Ziv complexity measures how frequently new patterns appear as a sequence unfolds. Basically how often new trends appear in the brainwaves/signals. We are primarily interested in the beta ΔLZC (task - resting):

- '>'0 = bad aptitude (task increases randomness a lot, hard to read)
- -0.005 to -0.015 = weak aptitude
- -0.015 to -0.030 = moderate aptitude
- '<'-0.030 = high aptitude

Low α-LZC → overly rigid idling rhythm → weak MI ERD (Alpha is a worse metric for us just because most of the EEG activity happening with hand movements is occuring in the Beta band)

High β-LZC → noisy motor inhibition → unstable control

For gap between real vs imagined, a gap of 0.02 or less would be considered good.

Good compatibility if:
- mean(beta ΔLZC_imagined) < −0.025
- mean(|beta gap|) < 0.02
- consistency across runs


In [ ]:
def lempel_ziv_complexity(binary_sequence: np.ndarray) -> float:
    # Compute normalized Lempel-Ziv complexity (LZ76).
    s = ''.join(binary_sequence.astype(str))
    n = len(s)
    i, k, l = 0, 1, 1
    c = 1
    while True:
        if s[i + k - 1] == s[l + k - 1]:
            k += 1
            if l + k > n:
                c += 1
                break
        else:
            if k > 1:
                i += 1
                k -= 1
            else:
                c += 1
                l += 1
                if l > n:
                    break
                i = 0
                k = 1
    return c * np.log2(n) / n

def binarize_signal(x: np.ndarray) -> np.ndarray:
    # Median-based binarization (robust to outliers).
    return (x > np.median(x)).astype(int)

def band_lzc_from_epochs(
    epochs: mne.Epochs,
    fmin: float,
    fmax: float
) -> np.ndarray:
    # Compute mean LZC per epoch, spatially averaged over channels.
    data = epochs.copy().filter(
        fmin, fmax,
        fir_design="firwin",
        verbose=False
    ).get_data()  # (epochs, channels, times)
    lzc_epochs = []
    for epoch in data:
        lzc_channels = []
        for ch_signal in epoch:
            binary = binarize_signal(ch_signal)
            lzc_channels.append(lempel_ziv_complexity(binary))
        lzc_epochs.append(np.mean(lzc_channels))  # spatial avg
    return np.array(lzc_epochs)

def lzc_modulation_band(
    task_epochs: mne.Epochs,
    baseline_raw: mne.io.BaseRaw,
    fmin: float,
    fmax: float
) -> float:
    # ΔLZC = Task − Rest
    baseline_epochs = mne.make_fixed_length_epochs(
        baseline_raw,
        duration=2.0,
        overlap=1.0,
        preload=True,
        verbose=False
    )
    LZC_task = band_lzc_from_epochs(task_epochs, fmin, fmax).mean()
    LZC_base = band_lzc_from_epochs(baseline_epochs, fmin, fmax).mean()
    return LZC_task - LZC_base


def imagined_real_lzc_features(
    analyzer: ImaginedVsActualAnalyzer,
    band: str = "beta",
    max_epochs: int = 20
) -> Dict[str, Dict[str, float]]:
    # Compute LZC features for imagined vs real movement.
    bands = {
        "alpha": (8, 13),
        "beta": (13, 30)
    }
    fmin, fmax = bands[band]
    baseline_raw = analyzer.data['baseline_eyes_open']['raw']
    results = {}
    for pair in analyzer.task_pairs:
        real_key = f"{pair['name']}_real"
        imag_key = f"{pair['name']}_imagined"
        if real_key not in analyzer.data or imag_key not in analyzer.data:
            continue
        real_epochs = analyzer.data[real_key]['epochs'][:max_epochs]
        imag_epochs = analyzer.data[imag_key]['epochs'][:max_epochs]

        if len(real_epochs) < 5 or len(imag_epochs) < 5:
            continue
        ΔLZC_real = lzc_modulation_band(
            real_epochs, baseline_raw, fmin, fmax
        )
        ΔLZC_imag = lzc_modulation_band(
            imag_epochs, baseline_raw, fmin, fmax
        )
        results[pair['name']] = {
            "ΔLZC_real": ΔLZC_real,
            "ΔLZC_imagined": ΔLZC_imag,
            "gap": ΔLZC_real - ΔLZC_imag
        }
    return results



analyzer = ImaginedVsActualAnalyzer(subject_id="S001")
analyzer.load_and_preprocess_data()
beta_lzc = imagined_real_lzc_features(analyzer, band="beta")
alpha_lzc = imagined_real_lzc_features(analyzer, band="alpha")
print("BETA LZC")
for k, v in beta_lzc.items():
    print(k, v)
print("\nALPHA LZC")
for k, v in alpha_lzc.items():
    print(k, v)

2026-01-20 02:29:43,820 - INFO - ======================================================================
2026-01-20 02:29:43,821 - INFO - LOADING DATA FOR S001 - IMAGINED VS ACTUAL ANALYSIS
2026-01-20 02:29:43,821 - INFO - ======================================================================
2026-01-20 02:29:43,822 - INFO - 
Loading baseline: eyes_open (Run 1)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


2026-01-20 02:29:43,848 - INFO -   ✓ Loaded baseline: 61.0s, 3 channels
2026-01-20 02:29:43,849 - INFO - 
Loading baseline: eyes_closed (Run 2)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


2026-01-20 02:29:43,867 - INFO -   ✓ Loaded baseline: 61.0s, 3 channels
2026-01-20 02:29:43,867 - INFO - 
Loading Left/Right Fist 1_real (Run 3)


...

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


2026-01-20 02:29:44,115 - INFO -   ✓ Loaded: 29 epochs, 3 channels
2026-01-20 02:29:44,115 - INFO - 
Loading Fists/Feet 3_imagined (Run 14)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


2026-01-20 02:29:44,137 - INFO -   ✓ Loaded: 29 epochs, 3 channels
2026-01-20 02:29:44,137 - INFO - 
✓ Successfully loaded 14 datasets


BETA LZC
Left/Right Fist 1 {'ΔLZC_real': np.float64(-0.03976927509557279), 'ΔLZC_imagined': np.float64(-0.035153167231540335), 'gap': np.float64(-0.004616107864032458)}
Fists/Feet 1 {'ΔLZC_real': np.float64(-0.039969975437487246), 'ΔLZC_imagined': np.float64(-0.054821800739156934), 'gap': np.float64(0.014851825301669688)}
Left/Right Fist 2 {'ΔLZC_real': np.float64(-0.051409894926611195), 'ΔLZC_imagined': np.float64(-0.039568574753658325), 'gap': np.float64(-0.01184132017295287)}
Fists/Feet 2 {'ΔLZC_real': np.float64(-0.03575526825728371), 'ΔLZC_imagined': np.float64(-0.04860009013980882), 'gap': np.float64(0.012844821882525115)}
Left/Right Fist 3 {'ΔLZC_real': np.float64(-0.0554239017649003), 'ΔLZC_imagined': np.float64(-0.04719518774640764), 'gap': np.float64(-0.008228714018492664)}
Fists/Feet 3 {'ΔLZC_real': np.float64(-0.05642740347447257), 'ΔLZC_imagined': np.float64(-0.053818299029584654), 'gap': np.float64(-0.002609104444887919)}

ALPHA LZC
Left/Right Fist 1 {'ΔLZC_real': np.floa